# Tutorial 2: Functions for Tensorflow Tutorial

In [1]:
#Tensorflow In-class tutorial from 10-11-2018

#!/usr/bin/env python2
# -*- coding: utf-8 -*-
"""
Created on Mon Oct  8 16:13:25 2018

@author: vganapa1
"""

%reset

import tensorflow as tf
import numpy as np

# fftshift implemented for Tensorflow (only for even dimensions)
def fftshift(mat2D, dim0, dim1): #fftshift == ifftshift when dimensions are all even
                                 #fftshift only works with even dimensions

    if (dim0==1) and (dim1==1):
        return mat2D    
    
    if (dim0%2) or (dim1%2):
        raise ValueError('Dimensions must be even to use fftshift.')

    dim0=tf.cast(dim0,tf.int32)
    dim1=tf.cast(dim1,tf.int32)

    piece1=tf.slice(mat2D,[0,0],[dim0//2,dim1//2])
    piece2=tf.slice(mat2D,[0,dim1//2],[dim0//2,dim1//2])
    piece3=tf.slice(mat2D,[dim0//2,0],[dim0//2,dim1//2])
    piece4=tf.slice(mat2D,[dim0//2,dim1//2],[dim0//2,dim1//2])

    top=tf.concat([piece4,piece3],axis=1)
    bottom=tf.concat([piece2,piece1],axis=1)

    final=tf.concat([top,bottom],axis=0)
    return final

def F(mat2D,dim0,dim1): # Fourier transform
    return fftshift(tf.fft2d(fftshift(mat2D, dim0, dim1)), dim0, dim1)

def Ft(mat2D,dim0,dim1): # Inverse Fourier transform
    return fftshift(tf.ifft2d(fftshift(mat2D, dim0, dim1)), dim0, dim1)

def H_fresnel_prop(u0,Nx,Ny,dx,dy,z,wavelength): # x is the row coordinate, y is the column coordinate

    Lx = Nx*dx
    Ly = Ny*dy
    k=1./wavelength #wavenumber
    
    fx=tf.linspace(-1/(2*dx),1/(2*dx)-1/Lx,Nx) #freq coords
    fy=tf.linspace(-1/(2*dy),1/(2*dy)-1/Ly,Ny) #freq coords
    
    FX,FY=tf.meshgrid(fx,fy, indexing = 'ij')
    FX_0 = tf.cast(FX,tf.complex64)
    FY_0 = tf.cast(FY,tf.complex64)
    H_fresnel=tf.exp(-1j*np.pi*tf.cast(wavelength,tf.complex64)*z*(FX_0**2+FY_0**2))
    
    zeros = tf.cast(tf.zeros([Nx,Ny]),dtype=tf.complex64)
    H_fresnel = tf.where(tf.sqrt(FX**2+FY**2)<=k,H_fresnel,zeros)
    
    return tf.abs(Ft(F(u0,Nx,Ny)*H_fresnel,Nx,Ny))**2    #Intensity

# Dense neural network layer
def dense_layer(input_layer,Nx,Ny,output_size,name='dense_layer'):
    
    input_layer = tf.reshape(input_layer, [1,-1])
    
    W = tf.get_variable('W_dense', \
                       initializer = tf.truncated_normal([Nx*Ny, output_size], stddev=0.01), \
                       dtype = tf.float32)
    
    b = tf.get_variable('b_dense', \
                        initializer = tf.truncated_normal([output_size,], stddev=0.01), \
                        dtype = tf.float32)
      
    output_layer = tf.matmul(input_layer,W) + b 
    
    return output_layer

# Convolutional neural network layer
def conv_layer(kernel_length, input_layer, input_channels, output_channels):

    kernel = tf.get_variable('kernel', \
                             initializer = tf.truncated_normal([kernel_length,kernel_length,input_channels,output_channels], \
                                                            mean=0, stddev=0.1, dtype=tf.float32), \
                             dtype = tf.float32)
    biases = tf.get_variable('biases', \
                             initializer = tf.truncated_normal([output_channels], mean=0, stddev=0.01, dtype=tf.float32), \
                             dtype = tf.float32)
    conv_layer = tf.nn.conv2d(input_layer, kernel, [1, 1, 1, 1], padding='SAME')
    conv_layer = tf.nn.bias_add(conv_layer, biases)
    conv_layer = tf.nn.relu(conv_layer)
    return conv_layer

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


# In-class exercise from 10-11-2018

In [2]:
### MY CODE ###
import tensorflow as tf
import numpy as np
from E30_Tensorflow_Wavelength_Functions import H_fresnel_prop, conv_layer, dense_layer

Nx = 2**5
Ny = 2**5
dx = 200e-9 #meters
dy = dx
z = 3e-6  #microns
kernel_length = 2**4
num_conv_layers = 5
learning_rate = 1e-2
num_iter = 1000

single_pt = np.zeros([Nx,Ny])
single_pt[Nx//2,Ny//2] = 1

with tf.Graph().as_default():
    wavelength = tf.placeholder(tf.float32)
    single_pt_0 = tf.constant(single_pt, dtype = tf.complex64)
    
    img_2D = H_fresnel_prop(single_pt_0,Nx,Ny,dx,dy,z,wavelength)
    
    img_2D = tf.expand_dims(img_2D, axis = 2)
    img_2D = tf.expand_dims(img_2D, axis = 0)
    input_layer = img_2D
    
    for i in range(num_conv_layers):
        with tf.variable_scope('cov_layer' + str(i)):
            output_layer = conv_layer(kernel_length, input_layer,1 ,1)
            input_layer = output_layer
    
    #Here we create two convolution layers
    #with tf.variable_scope('conv_layer1'):
        #output_layer = conv_layer(kernel_length, input_layer, 1, 1)
    
    #with tf.variable_scope('conv_layer2'):
        #output_layer = conv_layer(kernel_length, output_layer, 1, 1)
        
input_layer = dense_layer(input_layer, Nx, Ny, 1)
input_layer = tf.squeeze(input_layer, axis=0)
wavelegth_guess = input_layer[0]

error = (wavelength - wavelegth_guess)**2

optimizer = tf.train.AdamOptimizer(learning_rate)
train = optimizer.minimize(error)

init_op = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init_op)
    
    for i in range(num_iter):
        wavelength_0 = np.random.randint(450,750)*1e-9   #meters
        
        [_,error_0, wavelegth_1, img_2D_0] = sess.run([train, error, wavelength_guess, img_2D], feed_dict = {wavelength: wavelegth_0})
        
        print('iteration: ' + str(i))
        print('input wavelegth is: ' + str(wavelegth_0))
        print('output wavelegth is: ' + str(wavelegth_1))
        
        #plt.figure()
        #plt.plot(TO-DO)   #Plot the error
        #plt.show()
        #Plot it in log scale

ValueError: Tensor("W_dense:0", shape=(1024, 1), dtype=float32_ref) must be from the same graph as Tensor("Reshape:0", shape=(1, 1024), dtype=float32).

In [1]:
### PROF'S CODE ###
import tensorflow as tf
import numpy as np
from E30_Tensorflow_Wavelength_Functions import H_fresnel_prop, conv_layer, dense_layer

Nx = 2**5
Ny = 2**5
dx = 200e-9 #[m]
dy = dx
z = 3e-6
num_iter = 1000
kernel_length = 32
num_conv_layers = 5
#restore_model = False
learning_rate = 0.01

single_pt = np.zeros([Nx,Ny])
single_pt[Nx//2,Ny//2]=1

with tf.Graph().as_default():
    
    wavelength = tf.placeholder(tf.float32)
    single_pt_0 = tf.constant(single_pt,dtype=tf.complex64)
    

    img_2D = H_fresnel_prop(single_pt_0,Nx,Ny,dx,dy,z,wavelength)
                      
    img_2D = tf.expand_dims(img_2D, axis=2)
    img_2D = tf.expand_dims(img_2D, axis=0)

    input_layer = img_2D
    
    for i in range(num_conv_layers):
        with tf.variable_scope('conv_layer_' + str(i)):
            output_layer = conv_layer(kernel_length, input_layer, 1, 1)
            input_layer = output_layer
    
    input_layer = dense_layer(input_layer,Nx,Ny,1,name='dense_layer')
    input_layer = tf.squeeze(input_layer, axis=0)

    wavelength_guess = input_layer[0]
    MSE = (wavelength-wavelength_guess)**2 
    

    optimizer = tf.train.AdamOptimizer(learning_rate)
    train = optimizer.minimize(MSE)
    
    init_op = tf.global_variables_initializer()
#    saver = tf.train.Saver() 
        
    with tf.Session() as sess: 
        sess.run(init_op)
#        
#        if restore_model:
#            saver.restore(sess, 'model.ckpt')

        for i in range(num_iter):
            wavelength_0 = np.random.randint(450,750)*1e-9
#            wavelength_0 = 500*1e-9

            [_,MSE_0,wavelength_1,img_2D_0] = sess.run([train,MSE,wavelength_guess, img_2D],feed_dict = {wavelength: wavelength_0})
            print('MSE: ' + str(MSE_0))
            print(wavelength_0)
            print(wavelength_1)
            print
   
#        save_path = saver.save(sess, os.getcwd() + '/model.ckpt')
#        print("Model saved in file: %s" % save_path) 
        
#        plt.figure()
#        plt.imshow(PSF_0)
#        plt.colorbar()

MSE: 2.1287091e-05
5.02e-07
0.0046142954

MSE: 0.0080307685
5.42e-07
-0.08961401

MSE: 2.6391024e-06
6.52e-07
0.0016251835

MSE: 4.881134e-05
4.52e-07
0.0069869636

MSE: 0.0001191089
5.8e-07
0.010914283

MSE: 0.00018364079
6.57e-07
0.01355207

MSE: 0.00022580328
4.86e-07
0.015027238

MSE: 0.00023953106
7.42e-07
0.015477533

MSE: 0.0002265547
4.57e-07
0.015052191

MSE: 0.00019334716
6.93e-07
0.013905626

MSE: 0.00014862603
5.52e-07
0.01219178

MSE: 0.000101205536
5.71e-07
0.010060667

MSE: 5.861492e-05
6.83e-07
0.0076567214

MSE: 2.6183929e-05
7.2e-07
0.0051177433

MSE: 6.619554e-06
7e-07
0.0025735493

MSE: 2.0515701e-08
5.08e-07
0.00014374102

MSE: 4.268019e-06
4.58e-07
-0.0020654604

MSE: 1.5724554e-05
4.56e-07
-0.003964964

MSE: 3.0106792e-05
5.25e-07
-0.005486441

MSE: 4.3369837e-05
5.21e-07
-0.006585057

MSE: 5.2435124e-05
5.36e-07
-0.007240674

MSE: 5.562184e-05
6.89e-07
-0.007457316

MSE: 5.2733187e-05
6.87e-07
-0.007261075

MSE: 4.485526e-05
4.67e-07
-0.00669694

MSE: 3.3937275e

MSE: 7.3832775e-13
5.82e-07
-2.7726e-07

MSE: 9.144504e-13
5.87e-07
-3.6926903e-07

MSE: 9.639394e-13
6.69e-07
-3.1280425e-07

MSE: 7.025954e-13
7.2e-07
-1.1820961e-07

MSE: 2.3776939e-13
6.68e-07
1.8038392e-07

MSE: 2.0563617e-14
6.65e-07
5.215998e-07

MSE: 4.3138744e-14
6.43e-07
8.506987e-07

MSE: 1.3540921e-13
7.49e-07
1.1169799e-06

MSE: 6.9173687e-13
4.71e-07
1.3027072e-06

MSE: 5.7095046e-13
5.91e-07
1.3466126e-06

MSE: 2.77246e-13
7.47e-07
1.2735416e-06

MSE: 1.891011e-13
6.94e-07
1.1288575e-06

MSE: 9.491621e-14
6.25e-07
9.330847e-07

MSE: 1.2790591e-14
5.97e-07
7.100955e-07

MSE: 2.1957462e-14
6.4e-07
4.918195e-07

MSE: 4.4839522e-14
5.29e-07
3.1724653e-07

MSE: 1.7155602e-13
6.06e-07
1.918068e-07

MSE: 2.1201278e-13
6.02e-07
1.4155155e-07

MSE: 2.3294742e-13
6.49e-07
1.6635374e-07

MSE: 1.0403269e-13
5.85e-07
2.6245903e-07

MSE: 5.245236e-15
4.71e-07
3.98576e-07

MSE: 2.8643247e-15
4.79e-07
5.325194e-07

MSE: 1.165597e-15
6.11e-07
6.4514086e-07

MSE: 8.322739e-14
4.53e-07
7.4

MSE: 1.3667426e-15
6.85e-07
6.480305e-07

MSE: 3.675361e-15
5.61e-07
6.2162474e-07

MSE: 9.208125e-16
6.15e-07
5.846551e-07

MSE: 3.0648708e-14
7.33e-07
5.5793225e-07

MSE: 1.026643e-18
5.73e-07
5.719868e-07

MSE: 1.5113706e-15
5.46e-07
5.848764e-07

MSE: 1.1024743e-14
4.83e-07
5.879988e-07

MSE: 6.3731174e-15
4.88e-07
5.678318e-07

MSE: 3.6422166e-14
7.23e-07
5.321541e-07

MSE: 2.1055804e-14
6.87e-07
5.418938e-07

MSE: 2.5415366e-14
7.42e-07
5.82578e-07

MSE: 4.0947687e-14
4.52e-07
6.5435535e-07

MSE: 2.0808016e-16
6.6e-07
6.74425e-07

MSE: 2.2299106e-14
5.4e-07
6.8932883e-07

MSE: 3.1058231e-15
6.14e-07
6.697299e-07

MSE: 2.3322744e-14
4.87e-07
6.397179e-07

MSE: 6.050142e-15
5.01e-07
5.7878265e-07

MSE: 1.7802208e-14
6.4e-07
5.065751e-07

MSE: 1.5579257e-14
5.96e-07
4.711831e-07

MSE: 1.0588276e-14
5.7e-07
4.6710065e-07

MSE: 5.6459367e-14
7.24e-07
4.863882e-07

MSE: 1.3041545e-15
5.93e-07
5.568869e-07

MSE: 1.818642e-16
6.42e-07
6.285143e-07

MSE: 4.108525e-15
6.32e-07
6.960978e-07

MSE: 6.0468352e-15
7.16e-07
7.937614e-07

MSE: 3.5170428e-16
7.2e-07
7.012462e-07

MSE: 6.4238974e-15
5.43e-07
6.231492e-07

MSE: 4.1911993e-14
7.35e-07
5.302758e-07

MSE: 7.030834e-15
5.88e-07
5.041499e-07

MSE: 3.4563402e-15
5.63e-07
5.0420937e-07

MSE: 3.700025e-15
4.6e-07
5.208278e-07

MSE: 1.571434e-14
6.44e-07
5.1864316e-07

MSE: 3.4568312e-14
7.38e-07
5.520744e-07

MSE: 2.625697e-15
6.86e-07
6.3475846e-07

MSE: 5.6451776e-16
7e-07
7.2375957e-07

MSE: 8.504126e-15
7.05e-07
7.972178e-07

MSE: 7.787356e-15
7.49e-07
8.37246e-07

MSE: 2.536133e-14
6.89e-07
8.482524e-07

MSE: 4.6180524e-14
5.98e-07
8.1289653e-07

MSE: 6.5418963e-15
6.39e-07
7.19882e-07

MSE: 2.4681035e-17
6.18e-07
6.13032e-07

MSE: 2.2446725e-14
6.68e-07
5.1817767e-07

MSE: 5.688457e-14
7.14e-07
4.7549514e-07

MSE: 5.797655e-14
7.46e-07
5.052168e-07

MSE: 5.46139e-15
5.27e-07
6.009012e-07

MSE: 2.2482065e-14
5.16e-07
6.659402e-07

MSE: 1.3118618e-14
5.67e-07
6.815365e-07

MSE: 3.7134544e-14
4.7e-07
6.6270326e-07

MSE:

MSE: 3.6449466e-16
4.69e-07
4.8809176e-07

MSE: 3.7098996e-14
6.58e-07
4.6538898e-07

MSE: 3.1038384e-14
6.87e-07
5.108229e-07

MSE: 1.01410265e-16
6.02e-07
6.1207027e-07

MSE: 3.3030587e-16
7.18e-07
6.998257e-07

MSE: 9.127171e-14
4.83e-07
7.851121e-07

MSE: 1.8014548e-14
6.24e-07
7.582183e-07

MSE: 5.2381083e-14
4.59e-07
6.8786915e-07

MSE: 3.2120977e-14
7.25e-07
5.4577674e-07

MSE: 2.2777457e-19
4.79e-07
4.7947725e-07

MSE: 7.204464e-14
6.88e-07
4.1958867e-07

MSE: 1.7759007e-17
4.54e-07
4.5821415e-07

MSE: 3.8994983e-15
5.54e-07
4.9155403e-07

MSE: 4.1170405e-15
4.79e-07
5.4316416e-07

MSE: 2.1126882e-16
5.82e-07
5.674649e-07

MSE: 1.19654974e-14
4.85e-07
5.943869e-07

MSE: 1.5032823e-16
5.93e-07
5.8073914e-07

MSE: 5.6705557e-15
6.48e-07
5.7269693e-07

MSE: 8.071087e-16
6.2e-07
5.915903e-07

MSE: 1.7035498e-14
7.49e-07
6.184799e-07

MSE: 1.1467986e-14
5.81e-07
6.880887e-07

MSE: 6.012934e-16
6.89e-07
7.135213e-07

MSE: 6.1946256e-16
7.03e-07
7.27889e-07

MSE: 3.9883495e-15
6.69e-0

MSE: 9.619334e-17
5.89e-07
5.791922e-07

MSE: 1.9359074e-14
6.97e-07
5.5786313e-07

MSE: 1.5652364e-14
7.2e-07
5.948906e-07

MSE: 4.3202323e-14
4.71e-07
6.788517e-07

MSE: 2.1781827e-15
7.17e-07
6.70329e-07

MSE: 4.603691e-14
4.67e-07
6.8156214e-07

MSE: 1.1340866e-15
5.71e-07
6.046762e-07

MSE: 1.1716522e-14
6.3e-07
5.217571e-07

MSE: 1.23138e-14
6.02e-07
4.9103244e-07

MSE: 6.326146e-15
5.88e-07
5.084629e-07

MSE: 2.975011e-14
7.29e-07
5.565178e-07

MSE: 3.098375e-14
4.94e-07
6.70022e-07

MSE: 2.3484965e-15
7.49e-07
7.005387e-07

MSE: 7.828045e-14
4.68e-07
7.4778643e-07

MSE: 2.049631e-14
5.33e-07
6.761653e-07

MSE: 3.269433e-14
7.34e-07
5.531843e-07

MSE: 7.5346175e-17
5.25e-07
5.163198e-07

MSE: 4.0938993e-14
6.89e-07
4.866661e-07

MSE: 5.896665e-15
4.66e-07
5.4278973e-07

MSE: 1.6096668e-15
6.02e-07
5.6187935e-07

MSE: 1.5246185e-14
7.19e-07
5.9552457e-07

MSE: 5.0620605e-16
6.54e-07
6.7649904e-07

MSE: 1.758743e-17
7.36e-07
7.4019374e-07

